metric learningのためにJob DBラベルを付与

In [31]:
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sentence_transformers import SentenceTransformer

In [3]:
!ls data

jd_db.csv        job_database.csv


In [6]:
df = pd.read_csv('data/job_database.csv')
df.head()

,job_id,industry,role,rank,skill_sets
0,1,化学工業,安全管理者,シニア,- 長年の経験と知識を持った化学工業の安全管理に堪能すること\n- 法令や規制に基づいた安全...
1,2,物流・運輸業,物流コーディネーター,ミドル,- 物流業界の知識と経験\n- 日本語と英語の流暢なコミュニケーション能力\n- 計画と調整...
2,3,農業,物流担当,ミドル,- 農業物流の知識と経験\n- 貨物管理と配送計画の作成\n- 仕入れと倉庫管理のスキル\n...
3,4,物流・運輸業,倉庫管理,ジュニア,- 倉庫管理の基礎知識\n- 在庫管理の基本的な手続き\n- 品質管理の知識\n- 安全管理...
4,5,卸売業,受注管理,ジュニア,- 注文受付と確認の手続きに堪能な経験\n- 在庫管理と注文処理の知識\n- 商品情報の入力...


In [11]:
sentences = df.loc[:1, 'skill_sets']
sentences

0    - 長年の経験と知識を持った化学工業の安全管理に堪能すること\n- 法令や規制に基づいた安全...
1    - 物流業界の知識と経験\n- 日本語と英語の流暢なコミュニケーション能力\n- 計画と調整...
Name: skill_sets, dtype: object

In [12]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [13]:
%%time
embeddings = model.encode(sentences)

CPU times: user 1.01 s, sys: 504 ms, total: 1.51 s
Wall time: 6.53 s


In [15]:
embeddings.shape

(2, 384)

In [28]:
%%time
embeddings = model.encode(df['skill_sets'])
embeddings.shape

CPU times: user 914 ms, sys: 204 ms, total: 1.12 s
Wall time: 1.45 s


(500, 384)

In [30]:
embeddings

array([[ 0.08179791,  0.32798678, -0.26122308, ...,  0.05287066,
         0.2455722 , -0.00194317],
       [ 0.22747383, -0.11405208, -0.04640952, ...,  0.10698601,
        -0.02369653, -0.00276701],
       [ 0.2801222 ,  0.02454388,  0.06674293, ...,  0.15160972,
        -0.08498413,  0.03406944],
       ...,
       [ 0.17419872, -0.00867297,  0.06398754, ...,  0.2060584 ,
         0.06267846,  0.06713144],
       [-0.1771186 ,  0.0649301 , -0.24090846, ...,  0.08244245,
        -0.03545104,  0.01282372],
       [-0.10728465, -0.24596453, -0.13928963, ...,  0.1283184 ,
        -0.07387856, -0.05167005]], dtype=float32)

In [34]:
# PCA次元削減
pca = PCA(n_components=50)
reduced_data = pca.fit_transform(embeddings)
reduced_data.shape

(500, 50)

In [37]:
# k-meansクラスタリング
kmeans = KMeans(n_clusters=30, random_state=42)
kmeans.fit(reduced_data)
cluster_labels = kmeans.labels_
cluster_labels.shape

(500,)

In [39]:
pd.Series(cluster_labels).value_counts()

8     34
0     30
6     28
3     26
4     25
20    23
9     22
22    22
11    21
18    20
19    19
10    19
14    17
13    16
23    14
7     13
25    13
16    13
26    13
2     13
17    12
15    12
24    12
5     11
12    11
21    10
27    10
29     9
1      7
28     5
Name: count, dtype: int64

In [41]:
df['category_label'] = cluster_labels

In [44]:
df.to_csv('data/job_database_wCategory.csv', index=False)

In [52]:
df[df['category_label']==6]

,job_id,industry,role,rank,skill_sets,category_label
4,5,卸売業,受注管理,ジュニア,- 注文受付と確認の手続きに堪能な経験\n- 在庫管理と注文処理の知識\n- 商品情報の入力...,6
9,10,製薬業,購買担当,ミドル,- 薬品購買の経験\n- 価格負帳管理の経験\n- 供給鎖の整備と管理\n- 財務分析の能力...,6
22,23,卸売業,商品企画担当,ジュニア,- 新商品の開発経験\n- 市場調査能力\n- 競合分析能力\n- 製品開発プロセスに� f...,6
32,33,卸売業,受注管理,ジュニア,- 注文受付と確認の手続きに堪能な経験\n- 在庫管理と注文処理の知識\n- 商品情報の入力...,6
78,79,食品・飲料製造業,購買担当,シニア,- 5 年以上の購買経験\n- 食品・飲料製造業に特化した知識\n- 価格負債管理の経験\n...,6
97,98,卸売業,在庫管理,シニア,- 在庫管理の高度な知識と経験\n- 在庫システムの操作に堪能なITスキル\n- 財務分析や...,6
105,106,農業,購買担当,ミドル,- 農業品の購買経験\n- 仕入れ単価の計算能力\n- 仕入れニュースの把握\n- 販売予測...,6
120,121,化学工業,ロジスティクス担当,シニア,- 5年以上のロジスティクス業界の経験\n- 化学工業分野の知識と経験\n- 在庫管理と需要...,6
236,237,化学工業,購買担当,シニア,- 5年以上の購買経験\n- 化学工業分野の知識\n- 価格負帰分析の実践経験\n- 供給鎖...,6
241,242,製薬業,購買担当,ジュニア,- 購買関連の学業・専門知識\n- 仕入れ・仕入れ管理の経験\n- 価格評価能力\n- 数学...,6
